In [28]:
import numpy as np 
import pandas as pd 
import re 
import os 
import glob
from pathlib import Path
import collections
import numexpr
import multiprocessing
#import ray
#ray.init()
numexpr.set_num_threads(multiprocessing.cpu_count())

8

In [29]:
ROOT_PATH=os.path.abspath(os.curdir)
PATH_DIR_DATA="data/kara"
ALL_DIR_ON_DATA=glob.glob(f'{os.path.join(ROOT_PATH,PATH_DIR_DATA)}/*/')

ROOT_DIR_RAW=list(filter(re.compile(".*raw").match, ALL_DIR_ON_DATA))[0]
ROOT_DIR_BURNET=list(filter(re.compile(".*burnet").match, ALL_DIR_ON_DATA))[0]
ROOT_DIR_ZIPPED=list(filter(re.compile(".*zipped").match, ALL_DIR_ON_DATA))[0]
ALLXML_FILES = list(Path(ROOT_DIR_RAW).glob('**/*.xml'))

In [30]:
ALLXML_FILES[0]
ALLXML_FILES[0].name

'PPM-R15m-NE13143-2018_09_03-19_30.xml'

In [31]:
class Elements:
    XML_ELEMENTS={"NE":".//NE",
              "NEName":".//NE/NEIdentity/NEName",
              "NEType":".//NE/NEIdentity/NEType",
              "Entity":".//Entity",
              "Compliance":".//Entity/Details/Compliance",
              "TimeStamp":".//Entity/Details/TimeStamp",
              "EntityIdentity":".//Entity/EntityIdentity",
              "Counter":".//Entity/Details/Counter"}
    def __init__(self):
         for key, value in Elements.XML_ELEMENTS.items():
             setattr(self, key, value)   

    
    

    

In [32]:
elements = Elements()

In [33]:
NEName = pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.NEName,attrs_only=True)

NEType = pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.NEType)

Entity =pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.Entity,attrs_only=True)

Compliance = pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.Compliance)

TimeStamp = pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.TimeStamp,attrs_only=True)

Counter = pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.Counter)

EntityIdentity=pd.read_xml(ALLXML_FILES[0],
                        xpath=elements.EntityIdentity)

In [34]:
Entity=Entity.drop(columns=["sourceId","shelfId","equipId","emId"])
%time Entity["neId"]=int(Entity.neId.values[0].split("-")[-1])
#%time Entity['neId'].map(lambda row: row.neId.split("-")[-1])
Entity= Entity.rename(columns={"Id":"EntityId"})
Entity

CPU times: user 598 µs, sys: 0 ns, total: 598 µs
Wall time: 602 µs


,EntityId,neId
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143


In [35]:
re.findall(r'[A-Z][A-Z][0-9]*[0-9][^/]', NEName.longName.values[0])[0]

'MA0950'

In [36]:
NEName_val=re.findall(r'[A-Z][A-Z][0-9]*[0-9][^/]', NEName.longName.values[0])[0]
NEName_val
Entity.insert(2, 'NeAlias', NEName_val)
Entity

,EntityId,neId,NeAlias
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950


In [37]:
NEType_val=NEType.NEType.values[0]
NEType_val=NEType_val.replace(re.findall(r'[A-Z]*[A-Z][^0-9]', 
                              NEType_val)[0],f"{re.findall(r'[A-Z]*[A-Z][^0-9]', NEType_val)[0]} ")
NEType_val

'SPO 1410'

In [38]:
Entity.insert(3, 'NeType', NEType_val)
Entity

,EntityId,neId,NeAlias,NeType
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410


In [39]:
%time Entity=pd.concat([Entity,TimeStamp.rename(columns={"localTimeFormatId":"Time"})],axis=1)
Entity

CPU times: user 796 µs, sys: 71 µs, total: 867 µs
Wall time: 863 µs


,EntityId,neId,NeAlias,NeType,Time
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00


In [40]:
Compliance = Compliance.drop(columns=["complianceId","complianceType"]).rename(columns={"Compliance":"Failure"})
%time Entity=pd.concat([Entity,Compliance],axis=1)
Entity

CPU times: user 333 µs, sys: 31 µs, total: 364 µs
Wall time: 356 µs


,EntityId,neId,NeAlias,NeType,Time,Failure
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success


In [41]:
EntityIdentity = EntityIdentity.drop(columns=["entityTypeId"]).rename(columns={"EntityIdentity":"EntityType"})
%time Entity=pd.concat([Entity,EntityIdentity],axis=1)
Entity

CPU times: user 493 µs, sys: 0 ns, total: 493 µs
Wall time: 485 µs


,EntityId,neId,NeAlias,NeType,Time,Failure,EntityType
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON


In [42]:
Counter

,name,Counter
0,DropEvents,0.000000e+00
1,TotlOctets,2.083125e+11
2,TotPackets,6.359410e+08
3,BroadcPack,5.333000e+03
4,MulticPack,1.340900e+04
...,...,...
199,OvrSizPack,0.000000e+00
200,Fragments,0.000000e+00
201,Jabbers,0.000000e+00
202,Collisions,0.000000e+00


In [43]:
#%time Counter.name.apply(pd.value_counts).to_dict().keys()
%time count_names = list(collections.Counter(Counter.name.to_numpy()))
count_names

CPU times: user 180 µs, sys: 17 µs, total: 197 µs
Wall time: 203 µs


['DropEvents',
 'TotlOctets',
 'TotPackets',
 'BroadcPack',
 'MulticPack',
 'CrcAlgnErr',
 'UndSizPack',
 'OvrSizPack',
 'Fragments',
 'Jabbers',
 'Collisions',
 'Utilizatio']

In [44]:
%time list_filter_Counter = list(map(lambda name: Counter.query(f'name == "{name}"',engine="numexpr"), count_names))

CPU times: user 18.9 ms, sys: 251 µs, total: 19.2 ms
Wall time: 18.5 ms


In [45]:
list_filter_Counter[0]

,name,Counter
0,DropEvents,0.0
12,DropEvents,0.0
24,DropEvents,0.0
36,DropEvents,0.0
48,DropEvents,0.0
60,DropEvents,0.0
72,DropEvents,0.0
84,DropEvents,0.0
96,DropEvents,0.0
108,DropEvents,0.0


In [46]:
%time result_fiter= list(map(lambda name,df :df.rename({"Counter":name},axis=1).drop( ["name"], axis=1).reset_index().drop(['index'],axis=1) ,count_names,list_filter_Counter))
Counter=pd.concat(result_fiter,axis=1)

CPU times: user 17.5 ms, sys: 8.22 ms, total: 25.7 ms
Wall time: 23.8 ms


In [47]:
Counter

,DropEvents,TotlOctets,TotPackets,BroadcPack,MulticPack,CrcAlgnErr,UndSizPack,OvrSizPack,Fragments,Jabbers,Collisions,Utilizatio
0,0.0,2.083125e+11,635941045.0,5333.0,13409.0,0.0,0.0,0.0,0.0,0.0,0.0,19.6
1,0.0,9.620172e+09,71634941.0,3.0,990.0,0.0,0.0,134804.0,0.0,0.0,0.0,9.8
2,0.0,3.939395e+08,2539525.0,1.0,0.0,0.0,0.0,10044.0,0.0,0.0,0.0,0.3
3,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,8.028546e+09,60882891.0,12.0,990.0,0.0,0.0,156283.0,0.0,0.0,0.0,8.2
5,0.0,1.752692e+09,12597619.0,1.0,0.0,0.0,0.0,32805.0,0.0,0.0,0.0,1.7
6,0.0,8.293324e+09,62675626.0,4.0,994.0,0.0,0.0,65367.0,0.0,0.0,0.0,8.4
7,0.0,1.371270e+09,11957380.0,1.0,0.0,0.0,0.0,380.0,0.0,0.0,0.0,1.4
8,0.0,1.387770e+09,9991279.0,2.0,0.0,0.0,0.0,4851.0,0.0,0.0,0.0,1.4
9,0.0,1.131743e+09,9486460.0,0.0,991.0,0.0,0.0,13417.0,0.0,0.0,0.0,1.1


In [48]:
%time Entity=pd.concat([Entity,Counter],axis=1)
Entity

CPU times: user 0 ns, sys: 2.37 ms, total: 2.37 ms
Wall time: 2.03 ms


,EntityId,neId,NeAlias,NeType,Time,Failure,EntityType,DropEvents,TotlOctets,TotPackets,BroadcPack,MulticPack,CrcAlgnErr,UndSizPack,OvrSizPack,Fragments,Jabbers,Collisions,Utilizatio
0,EN-0-13143-0-0-3-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,2.083125e+11,635941045.0,5333.0,13409.0,0.0,0.0,0.0,0.0,0.0,0.0,19.6
1,EN-0-13143-0-0-4-9-2-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,9.620172e+09,71634941.0,3.0,990.0,0.0,0.0,134804.0,0.0,0.0,0.0,9.8
2,EN-0-13143-0-0-4-9-10-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,3.939395e+08,2539525.0,1.0,0.0,0.0,0.0,10044.0,0.0,0.0,0.0,0.3
3,EN-0-13143-0-0-4-9-1-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EN-0-13143-0-0-4-9-4-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,8.028546e+09,60882891.0,12.0,990.0,0.0,0.0,156283.0,0.0,0.0,0.0,8.2
5,EN-0-13143-0-0-4-9-5-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.752692e+09,12597619.0,1.0,0.0,0.0,0.0,32805.0,0.0,0.0,0.0,1.7
6,EN-0-13143-0-0-4-9-3-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,8.293324e+09,62675626.0,4.0,994.0,0.0,0.0,65367.0,0.0,0.0,0.0,8.4
7,EN-0-13143-0-0-4-9-6-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.371270e+09,11957380.0,1.0,0.0,0.0,0.0,380.0,0.0,0.0,0.0,1.4
8,EN-0-13143-0-0-4-9-7-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.387770e+09,9991279.0,2.0,0.0,0.0,0.0,4851.0,0.0,0.0,0.0,1.4
9,EN-0-13143-0-0-4-9-9-0-0-0-0-21-0,13143,MA0950,SPO 1410,2018-09-03 19.30.00,Success,RMON,0.0,1.131743e+09,9486460.0,0.0,991.0,0.0,0.0,13417.0,0.0,0.0,0.0,1.1


In [49]:
%time ElementID = Entity.apply(lambda raw : f"{raw.NeType}_{raw.EntityType}({raw.EntityId})".replace(" ","_") ,axis=1)
%time Entity=pd.concat([Entity,ElementID],axis=1)
%Entity.columns = [*Entity.columns[:-1], 'ElementID']
Entity

CPU times: user 2.21 ms, sys: 0 ns, total: 2.21 ms
Wall time: 2.22 ms
CPU times: user 766 µs, sys: 0 ns, total: 766 µs
Wall time: 748 µs


UsageError: Line magic function `%Entity.columns` not found.


In [24]:
Entity.columns

Index(['EntityId', 'neId', 'NeAlias', 'NeType', 'Time', 'Failure',
       'EntityType', 'DropEvents', 'TotlOctets', 'TotPackets', 'BroadcPack',
       'MulticPack', 'CrcAlgnErr', 'UndSizPack', 'OvrSizPack', 'Fragments',
       'Jabbers', 'Collisions', 'Utilizatio', 'ElementID'],
      dtype='object')

In [25]:
%time Entity.to_csv(ROOT_DIR_ZIPPED+ALLXML_FILES[0].name.replace("xml","csv"),index=False)

CPU times: user 4.27 ms, sys: 74 µs, total: 4.35 ms
Wall time: 3.81 ms


In [27]:
Entity.dtypes

EntityId       object
neId           object
NeAlias        object
NeType         object
Time           object
Failure        object
EntityType     object
DropEvents    float64
TotlOctets    float64
TotPackets    float64
BroadcPack    float64
MulticPack    float64
CrcAlgnErr    float64
UndSizPack    float64
OvrSizPack    float64
Fragments     float64
Jabbers       float64
Collisions    float64
Utilizatio    float64
ElementID      object
dtype: object